<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

FileNotFoundError: ignored

In [ ]:
display(data.info())
display(data.head())

**Заполним пропуски в столбце Tenure (сколько лет человек является клиентом банка) - если нет значения, то скорее всего человек является клиентом банка менее года - значит, значение может быть равно "0"**

In [ ]:
data['Tenure'] = data['Tenure'].fillna(0)

In [ ]:
data.info()

**Определим категорийные столбцы методом nunique**

In [ ]:
data.nunique()

**Проверим данные на мультиколлинерность построением графика scatter_matrix**

In [ ]:
pd.plotting.scatter_matrix(data,figsize=(12,12))
plt.show

**На диаграммах не зафиксированна мультиколлинерность данных, поэтому используем все интересующие нас столбцы**

In [ ]:
target = data['Exited']
features = data.drop(['Exited', 'RowNumber', 'CustomerId', 'Gender', 'Surname', 'Geography'], axis=1)
display(target.head())
display(features.head())

In [ ]:
features_train, features_test, target_train, target_test = (
    train_test_split(
        features, target, test_size=0.2, random_state=12345, stratify=target
    )
)

In [ ]:
features_train, features_valid, target_train, target_valid = (
    train_test_split(
        features_train, target_train, test_size=0.2, random_state=12345, stratify=target_train
    )
)

**Проверим разбивку по их длине**

In [ ]:
len(features_train)

In [ ]:
len(features_test)

In [ ]:
len(features_valid)

## Исследование задачи

**Проверим полученную выборку на дисбаланс классов, для этого выведем на экран крайние значения столбцов методом describe**

In [ ]:
features.describe()

**Как видно из полученной таблицы - у нас наблюдается дисбаланс классов.**

**Для начала посчитаем метрики на несбалансированных классах.**

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print("F1:", f1_score(target_valid, predicted_valid),'\n',
      "AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None))

In [ ]:
best_model_f1 = None
best_model_roc_auc = None
best_result_f1 = 0
best_result_roc_auc = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train) 
    predictions_valid = model_tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    if result_f1 > best_result_f1:
        best_model_f1 = model_tree
        best_result_f1 = result_f1
        best_depth_f1 = depth
    probabilities_valid = model_tree.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    result_roc_auc = roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None)
    if result_roc_auc > best_result_roc_auc:
        best_model_roc_auc = model_tree
        best_result_roc_auc = result_roc_auc
        best_depth_roc_auc = depth
print(
    "F1 лучшей модели:", best_result_f1, '\n',
    "Глубина дерева:", best_depth_f1, '\n',
    "AUC-ROC лучшей модели:", best_result_roc_auc, '\n',
    "Глубина дерева:", best_depth_roc_auc,
    )

In [ ]:
best_model_f1 = None
best_model_roc_auc = None
best_result_f1 = 0
best_result_roc_auc = 0
for est in range(1, 81, 10):
    for depth in range (1, 21):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predictions_valid = model_forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        if result_f1 > best_result_f1:
            best_model_f1 = model_forest
            best_result_f1 = result_f1
            best_depth_f1 = depth
            best_est_f1 = est
        probabilities_valid = model_forest.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        result_roc_auc = roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None)
        if result_roc_auc > best_result_roc_auc:
            best_model_roc_auc = model_forest
            best_result_roc_auc = result_roc_auc 
            best_depth_roc_auc = depth
            best_est_roc_auc = est
print(
    "F1 лучшей модели:", best_result_f1, '\n',
    "Количество деревьев:", best_est_f1, '\n',
    "Глубина дерева:", best_depth_f1, '\n',
    "AUC-ROC лучшей модели:", best_result_roc_auc, '\n',
    "Количество деревьев:", best_est_roc_auc, '\n',
    "Глубина дерева:", best_depth_roc_auc)

**Наилучшие результаты у модели "Случайный лес" -**

**F1: 0.533,**

**AUC-ROC: 0.831**

## Борьба с дисбалансом

**Избавимся от дисбаланса класса методом увеличения и уменьшения выборки**

**Увеличим выборку методом upsample**

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

**Увеличим обучающую выборку:**

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

**Проверим размер выборке после увеличения:**

In [ ]:
features_zeros = features_upsampled[target_upsampled == 0]
features_ones = features_upsampled[target_upsampled == 1]
target_zeros = target_upsampled[target_upsampled == 0]
target_ones = target_upsampled[target_upsampled == 1]

In [ ]:
features_zeros.shape[0]

In [ ]:
features_ones.shape[0]

**Уменьшим выборку методом downsample**

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

**Уменьшим обучающую выборку:**

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

**Проверим размер выборке после уменьшения:**

In [ ]:
features_zeros = features_downsampled[target_downsampled == 0]
features_ones = features_downsampled[target_downsampled == 1]
target_zeros = target_downsampled[target_downsampled == 0]
target_ones = target_downsampled[target_downsampled == 1]

In [ ]:
features_zeros.shape[0]

In [ ]:
features_ones.shape[0]

**Обучим и проверим модели на сбалансированных данных**

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print("F1:", f1_score(target_valid, predicted_valid),'\n',
      "AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None))

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print("F1:", f1_score(target_valid, predicted_valid),'\n',
      "AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None))

In [ ]:
best_model_f1 = None
best_model_roc_auc = None
best_result_f1 = 0
best_result_roc_auc = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_downsampled, target_downsampled)
    predictions_valid = model_tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    if result_f1 > best_result_f1:
        best_model_f1 = model_tree
        best_result_f1 = result_f1
        best_depth_f1 = depth
    probabilities_valid = model_tree.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    result_roc_auc = roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None)
    if result_roc_auc > best_result_roc_auc:
        best_model_roc_auc = model_tree
        best_result_roc_auc = result_roc_auc
        best_depth_roc_auc = depth
print(
    "F1 лучшей модели:", best_result_f1, '\n',
    "Глубина дерева:", best_depth_f1, '\n',
    "AUC-ROC лучшей модели:", best_result_roc_auc, '\n',
    "Глубина дерева:", best_depth_roc_auc,
    )

In [ ]:
best_model_f1 = None
best_model_roc_auc = None
best_result_f1 = 0
best_result_roc_auc = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_upsampled, target_upsampled)
    predictions_valid = model_tree.predict(features_valid)
    result_f1 = f1_score(target_valid, predictions_valid)
    if result_f1 > best_result_f1:
        best_model_f1 = model_tree
        best_result_f1 = result_f1
        best_depth_f1 = depth
    probabilities_valid = model_tree.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    result_roc_auc = roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None)
    if result_roc_auc > best_result_roc_auc:
        best_model_roc_auc = model_tree
        best_result_roc_auc = result_roc_auc
        best_depth_roc_auc = depth
print(
    "F1 лучшей модели:", best_result_f1, '\n',
    "Глубина дерева:", best_depth_f1, '\n',
    "AUC-ROC лучшей модели:", best_result_roc_auc, '\n',
    "Глубина дерева:", best_depth_roc_auc,
    )

In [ ]:
best_model_f1 = None
best_model_roc_auc = None
best_result_f1 = 0
best_result_roc_auc = 0
for est in range(1, 121, 10):
    for depth in range (1, 31):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_downsampled, target_downsampled)
        predictions_valid = model_forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        if result_f1 > best_result_f1:
            best_model_f1 = model_forest
            best_result_f1 = result_f1
            best_depth_f1 = depth
            best_est_f1 = est
        probabilities_valid = model_forest.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        result_roc_auc = roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None)
        if result_roc_auc > best_result_roc_auc:
            best_model_roc_auc = model_forest
            best_result_roc_auc = result_roc_auc 
            best_depth_roc_auc = depth
            best_est_roc_auc = est
print(
    "F1 лучшей модели:", best_result_f1, '\n',
    "Количество деревьев:", best_est_f1, '\n',
    "Глубина дерева:", best_depth_f1, '\n',
    "AUC-ROC лучшей модели:", best_result_roc_auc, '\n',
    "Количество деревьев:", best_est_roc_auc, '\n',
    "Глубина дерева:", best_depth_roc_auc)

In [ ]:
best_model_f1 = None
best_model_roc_auc = None
best_result_f1 = 0
best_result_roc_auc = 0
for est in range(1, 181, 10):
    for depth in range (1, 31):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_upsampled, target_upsampled)
        predictions_valid = model_forest.predict(features_valid)
        result_f1 = f1_score(target_valid, predictions_valid)
        if result_f1 > best_result_f1:
            best_model_f1 = model_forest
            best_result_f1 = result_f1
            best_depth_f1 = depth
            best_est_f1 = est
        probabilities_valid = model_forest.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        result_roc_auc = roc_auc_score(target_valid, probabilities_one_valid, average="macro", sample_weight=None)
        if result_roc_auc > best_result_roc_auc:
            best_model_roc_auc = model_forest
            best_result_roc_auc = result_roc_auc 
            best_depth_roc_auc = depth
            best_est_roc_auc = est
print(
    "F1 лучшей модели:", best_result_f1, '\n',
    "Количество деревьев:", best_est_f1, '\n',
    "Глубина дерева:", best_depth_f1, '\n',
    "AUC-ROC лучшей модели:", best_result_roc_auc, '\n',
    "Количество деревьев:", best_est_roc_auc, '\n',
    "Глубина дерева:", best_depth_roc_auc)

## Тестирование модели

In [ ]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=101, max_depth=11)
model_forest.fit(features_upsampled, target_upsampled)
predictions_test = model_forest.predict(features_test)
result_f1 = f1_score(target_test, predictions_test)
probabilities_test = model_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
result_roc_auc = roc_auc_score(target_test, probabilities_one_test, average="macro", sample_weight=None)
print(
    "F1 лучшей модели:", result_f1, '\n',   
    "AUC-ROC лучшей модели:", result_roc_auc)

**Проверим модель на адекватность с помощью DummyClassifier.**

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_clf.fit(features_upsampled, target_upsampled)
predictions_dummy_test = dummy_clf.predict(features_test)
print(
    'dummy_score:', dummy_clf.score(features_test, target_test), '\n',
    'f1_score:', f1_score(target_test, predictions_dummy_test)
)

**Для понимания, какие факторы влияют на работу модели, воспользуемся методом feature_importances_ и выведем график**

In [ ]:
feature_importances = pd.Series(model_forest.feature_importances_, index=features.columns)
feature_importances.nlargest(25).plot(kind='barh')
plt.show

**Наиболее слабыми признаками являются "наличие кредитной карты", "активность клиента" и "сколько лет человек является клиентом банка"**

**Построим ROC-кривую по результатам предсказания модели**

In [ ]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=21, max_depth=7)
model_forest.fit(features_upsampled, target_upsampled)

probabilities_test = model_forest.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_test)

plt.figure()

plt.plot(fpr, tpr, linestyle='--')
plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlim([0, 1])
plt.ylim([0, 1])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('ROC-кривая')
plt.show()

**Выведем средние значения тестовой и обучающей выборки по целевому признаку "Exited"**

In [ ]:
print(
'Среднее значение целевого признака:', target.mean(), '\n',
'Среднее значение предсказаний тестовой модели:', probabilities_one_test.mean())

# Вывод

**В данном проекте мы рассмотрели работу нескольких обучающих моделей - наилучший показатель у модели "Случайный лес" - F1 тестовой модели: 0.593, AUC-ROC тестовой модели: 0.843.**

**Если сравнивать в количественном выражении работу модели, то она далека от идеала, поскольку среднее значение целевого признака исходной выборки составляет 0.2037, а среднее значение предсказаний тестовой модели составляет 0.412.**

**Но это также может говорить о том, что риск ухода клиента возрастает и менеджерам банка необходимо предпринимать действия для удержания клиентов.**

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*